## Ran a bunch of queries to examine encounter types, and count the frequencies.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime
from impala.util import as_pandas
%matplotlib notebook
plt.style.use('ggplot')

In [2]:
from impala.dbapi import connect
conn = connect(host="socbddn01.sharp.com", port=21050)
cur = conn.cursor()
cur.execute("use poc_sandbox")

In [19]:
query_admit_RRT = "SELECT enc.admit_type_cd, cv_admit_type.description AS admit_type_des, count(1)\
FROM encounter enc \
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value \
WHERE enc.encntr_id IN (SELECT encntr_id \
                    FROM clinical_event \
                    WHERE event_cd = '54411998' \
                    AND result_status_cd NOT IN ('31', '36') \
                    AND valid_until_dt_tm > 4000000000000 \
                    AND event_class_cd not in ('654645')) \
AND (enc.encntr_complete_dt_tm < 4000000000000) \
AND enc.loc_facility_cd='633867' \
GROUP BY enc.admit_type_cd, admit_type_des;"
cur.execute(query_admit_RRT)
cur.fetchall()



[('670839', 'Urgent', 311),
 ('309203', 'Emergency', 262),
 ('0', None, 37),
 ('670840', 'Elective', 82)]

In [20]:
query_admit_NotRRT = "SELECT enc.admit_type_cd, cv_admit_type.description AS admit_type_des, count(1)\
FROM encounter enc \
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value \
WHERE enc.encntr_id NOT IN (SELECT encntr_id \
                    FROM clinical_event \
                    WHERE event_cd = '54411998' \
                    AND result_status_cd NOT IN ('31', '36') \
                    AND valid_until_dt_tm > 4000000000000 \
                    AND event_class_cd not in ('654645')) \
AND (enc.encntr_complete_dt_tm < 4000000000000) \
AND enc.loc_facility_cd='633867' \
GROUP BY enc.admit_type_cd, admit_type_des;"
cur.execute(query_admit_NotRRT)
cur.fetchall()



[('4659953', 'Delivery, Unscheduled', 9),
 ('670839', 'Urgent', 5023),
 ('4659954', 'Newborn GT 24Hrs Old', 1),
 ('309203', 'Emergency', 3917),
 ('4659952', 'Delivery, Scheduled', 1),
 ('0', None, 229671),
 ('670840', 'Elective', 2449)]

In [22]:
query_count_NotRRT = "SELECT count(1) FROM \
(SELECT enc.admit_type_cd \
FROM encounter enc \
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value \
WHERE enc.encntr_id NOT IN (SELECT encntr_id \
                    FROM clinical_event \
                    WHERE event_cd = '54411998' \
                    AND result_status_cd NOT IN ('31', '36') \
                    AND valid_until_dt_tm > 4000000000000 \
                    AND event_class_cd not in ('654645')) \
AND (enc.encntr_complete_dt_tm < 4000000000000) \
AND enc.loc_facility_cd='633867') t;"
# AND enc.admit_type_cd!='0') t ;"
cur.execute(query_count_NotRRT)
cur.fetchall()



[(241071,)]

In [28]:
query_enctypecl_RRT = "SELECT enc.encntr_type_class_cd , cv_enc_type_class.description AS enc_type_class_desc, count(1)\
FROM encounter enc \
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value \
WHERE enc.encntr_id IN (SELECT encntr_id \
                    FROM clinical_event \
                    WHERE event_cd = '54411998' \
                    AND result_status_cd NOT IN ('31', '36') \
                    AND valid_until_dt_tm > 4000000000000 \
                    AND event_class_cd not in ('654645')) \
AND (enc.encntr_complete_dt_tm < 4000000000000) \
AND enc.loc_facility_cd='633867' \
GROUP BY enc.encntr_type_class_cd, enc_type_class_desc;"
cur.execute(query_enctypecl_RRT)
cur.fetchall()

[('391', 'Inpatient', 670),
 ('393', 'Outpatient', 3),
 ('392', 'Observation', 19)]

In [38]:
query_enctypecl_RRT_admit = "SELECT enc.admit_type_cd, cv_admit_type.description AS admit_type_des, count(1)\
FROM encounter enc \
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value \
WHERE enc.encntr_id IN (SELECT encntr_id \
                    FROM clinical_event \
                    WHERE event_cd = '54411998' \
                    AND result_status_cd NOT IN ('31', '36') \
                    AND valid_until_dt_tm > 4000000000000 \
                    AND event_class_cd not in ('654645')) \
AND (enc.encntr_complete_dt_tm < 4000000000000) \
AND enc.loc_facility_cd='633867' \
AND enc.encntr_type_class_cd='391'\
GROUP BY enc.admit_type_cd, admit_type_des;"
cur.execute(query_enctypecl_RRT_admit)
cur.fetchall()

#AND enc.admit_type_cd!='0'  -> 655 results, all inpatient.\
# only "AND enc.encntr_type_class_cd='391'\" -> 670 results, 15 of which have admit_type_cd=0

[('670839', 'Urgent', 311),
 ('309203', 'Emergency', 262),
 ('0', None, 15),
 ('670840', 'Elective', 82)]

In [29]:
query_enctype_NotRRT = "SELECT enc.encntr_type_class_cd , cv_enc_type_class.description AS enc_type_class_desc, count(1)\
FROM encounter enc \
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value \
WHERE enc.encntr_id NOT IN (SELECT encntr_id \
                    FROM clinical_event \
                    WHERE event_cd = '54411998' \
                    AND result_status_cd NOT IN ('31', '36') \
                    AND valid_until_dt_tm > 4000000000000 \
                    AND event_class_cd not in ('654645')) \
AND (enc.encntr_complete_dt_tm < 4000000000000) \
AND enc.loc_facility_cd='633867' \
GROUP BY enc.encntr_type_class_cd, enc_type_class_desc;"
cur.execute(query_enctype_NotRRT)
cur.fetchall()

[('391', 'Inpatient', 12907),
 ('393', 'Outpatient', 209944),
 ('0', None, 1),
 ('397', 'Recurring', 12124),
 ('392', 'Observation', 6095)]

In [30]:
query_enctype_NotRRT_admit = "SELECT enc.encntr_type_class_cd , cv_enc_type_class.description AS enc_type_class_desc, count(1)\
FROM encounter enc \
LEFT OUTER JOIN code_value cv_admit_type ON enc.admit_type_cd = cv_admit_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type ON enc.encntr_type_cd = cv_enc_type.code_value \
LEFT OUTER JOIN code_value cv_enc_type_class ON enc.encntr_type_class_cd = cv_enc_type_class.code_value \
WHERE enc.encntr_id NOT IN (SELECT encntr_id \
                    FROM clinical_event \
                    WHERE event_cd = '54411998' \
                    AND result_status_cd NOT IN ('31', '36') \
                    AND valid_until_dt_tm > 4000000000000 \
                    AND event_class_cd not in ('654645')) \
AND (enc.encntr_complete_dt_tm < 4000000000000) \
AND enc.loc_facility_cd='633867' \
AND enc.admit_type_cd!='0'\
GROUP BY enc.encntr_type_class_cd, enc_type_class_desc;"
cur.execute(query_enctype_NotRRT_admit)
cur.fetchall()

# adding in that AND enc.admit_type_cd!='0' clause makes the distros of enctr_type_class_cd match 

[('391', 'Inpatient', 11388),
 ('393', 'Outpatient', 2),
 ('392', 'Observation', 10)]